In [3]:
import pandas as pd
import altair as alt
import json
import geopandas as gpd
import numpy as np

### Load data

In [8]:
df = pd.read_csv("../data/clean_number-of-deaths-by-risk-factor.csv")

In [10]:
#source: https://www.naturalearthdata.com/downloads/110m-cultural-vectors/
#source:https://towardsdatascience.com/a-complete-guide-to-an-interactive-geographical-map-using-python-f4c5197e23e0
shapefile = '../data/geographic_data/ne_110m_admin_0_countries.shp'
gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
gdf.columns = ['country', 'country_code', 'geometry']
gdf.head()

,country,country_code,geometry
0,Fiji,FJI,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,United Republic of Tanzania,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,Western Sahara,SAH,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Canada,CAN,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,United States of America,USA,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


### Data Wrangling

In [11]:
df1 = df.query("year == 2017")
df1.drop(['country'], axis =1 ,inplace = True)

/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
geo_df = gdf.merge(df1, left_on = 'country_code', right_on = 'code')

In [13]:
geo_df.drop(['code'], axis =1 ,inplace = True)

In [14]:
geo_df.iloc[:,4:] = geo_df.iloc[:,4:].div(geo_df.iloc[:,4:].sum(axis=1), axis=0)

In [15]:
geo_df.head()

,country,country_code,geometry,year,unsafe_water_source,poor_sanitation,no_access_to_handwashing_facility,indoor_air_pollution,non-exclusive_breastfeeding,discontinued_breastfeeding,...,high_blood_sugar,obesity,high_blood_pressure,smoking,iron_deficiency,zinc_deficiency,vitamin-a_deficiency,low_bone_mineral_density,air_pollution_outdoor_&_indoor,outdoor_air_pollution
0,Fiji,FJI,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",2017,0.005196,0.002336,0.001356,0.036200,0.000660,0.000061,...,0.243027,0.201757,0.157771,0.083694,0.000298,0.000170,0.000920,0.000927,0.063239,0.027052
1,United Republic of Tanzania,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",2017,0.044584,0.033470,0.042147,0.080074,0.006113,0.000276,...,0.080928,0.044226,0.113546,0.054567,0.005559,0.001634,0.009364,0.004113,0.097472,0.017846
2,Canada,CAN,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",2017,0.000223,0.000285,0.000429,0.000085,0.000056,0.000006,...,0.170675,0.153391,0.198189,0.230628,0.000018,0.000004,0.000017,0.014150,0.036608,0.037084
3,United States of America,USA,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",2017,0.000123,0.000239,0.000330,0.000211,0.000080,0.000015,...,0.174219,0.169454,0.188320,0.174917,0.000062,0.000002,0.000012,0.008423,0.044560,0.045440
4,Kazakhstan,KAZ,"POLYGON ((87.35997 49.21498, 86.59878 48.54918...",2017,0.000252,0.000248,0.000497,0.012750,0.000460,0.000013,...,0.133269,0.141913,0.234885,0.127893,0.000091,0.000065,0.000290,0.001488,0.050941,0.038800


In [17]:
geo_df.to_file("../data/map_data.geojson", driver='GeoJSON')